In [60]:
import sqlite3
# File with functions
from p2func import show_db_tables

In [65]:
sqlitefile = 't_homechef.sqlite'
sqlitefile = 't_csv_meals.sqlite'

In [66]:
# Creation of ingredients table
create_scratch_ingredients_table(sqlitefile)

# Creation of table meal_info
create_scratch_meal_info_table(sqlitefile)

# Creation of table meal_bow
# Meal_Bow is a table in recipes.sqlite will have all the words that refer to: 
# - Measure Units, 
# - Fractions, 
# - Ingredient texture types, 
# - type of ingredient
create_scratch_meal_bow_table(sqlitefile)

# Insertion of Bag of words into a sql table
fraction_list = ['½', '⅓', '⅔', '¼', '¾', '⅕', '⅖', '⅗', '⅘', '⅙', '⅚', '⅐', '⅛', '⅜', '⅝', '⅞', '⅑', '⅒']
measure_units = ['cups','cup','cloves','clove','fl\.','oz\.','head','sprigs', 'sprig','tsp.','tbsp.']
shape_ing = ['breasts','cubed', 'chops','chopped','chunks','crumbles','florets', 'flakes', 'glaze', 'halves', 'hearts', 
             'kernels','package','paste', 'preserves','pesto','powder','rolls','roll', 
             'seasoning', 'seeds', 'sauce', 'slices','shaved','slivered','with black spots']
type_ing = ['ripe','shredded','bone-in','boneless','skinless']

# BAG OF WORDS
# Meal_Bow is a table in recipes.sqlite will have all the words that refer to: 
# - Measure Units, 
# - Fractions, 
# - Ingredient texture types, 
# - type of ingredient
# 
fill_bow(sqlitefile, fraction_list, measure_units, shape_ing, type_ing)

## Confirm all tables have been created

In [67]:
show_db_tables(sqlitefile)

['meal_ingredients', 'meal_info', 'meal_bow']


### Functions that couldnt get imported from p2func.py

create_scratch_meal_info_table(sqlitefile)
    Creates a table with the columns
    required by meal_info in the sqlitefile database

create_scratch_ingredients_table(sqlitefile)
    Creates a table with the columns
    required by ingredients table in the sqlitefile database

create_scratch_meal_bow_table(sqlitefile)
    Creates a table with the columns
    required by meal_bow table in the sqlitefile database
    
fill_bow(sqlitefile, fraction_list, measure_units, shape_ing, type_ing)
    Fills the meal_bow sql table with the items in:
    - fraction_list
    - measure_units
    - shape_ing
    - type_ing

In [62]:
def create_scratch_meal_info_table(sqlitefile):
    table_name           = 'meal_info'   # name of the table to be created
    id_field             = 'meal_id' # name of the ID column
    title_col            = 'title'
    rating_col           = 'rating'     # integer counting the starts [0,5]
    duration_col         = 'duration'   # text for now -> integer counting time in minutes
    expiration_col       = 'expiration' # Integer counting in days
    expertise_col        = 'exp_level'  # Text -> Categorical
    spicy_col            = 'spiciness'  # Text -> Categorical
    cat_col              = 'category'   # Text -> Categorical
    field_type_1         = 'INTEGER'
    field_type_2         = 'TEXT'  # column data type
    conn = sqlite3.connect(sqlitefile)
    c = conn.cursor()
    try:
    # Create table with word column as primary key
        c.execute('CREATE TABLE {tn} ({nf} {ft} PRIMARY KEY)'\
                .format(tn=table_name, nf=id_field, ft=field_type_1))
        c.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}"\
                .format(tn=table_name, cn=title_col, ct=field_type_2))
        c.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}"\
                .format(tn=table_name, cn=rating_col, ct=field_type_1))
        c.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}"\
                .format(tn=table_name, cn=duration_col, ct=field_type_2))
        c.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}"\
                .format(tn=table_name, cn=expiration_col, ct=field_type_1))
        c.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}"\
                .format(tn=table_name, cn=expertise_col, ct=field_type_2))
        c.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}"\
                .format(tn=table_name, cn=spicy_col, ct=field_type_2))
        c.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}"\
                .format(tn=table_name, cn=cat_col, ct=field_type_2))
    except sqlite3.OperationalError as oe:
        print(oe)
    conn.commit()
    conn.close()

def create_scratch_ingredients_table(sqlitefile):
    table_name  = 'meal_ingredients'  # name of the table to be created
    id_column   = 'meal_id'           # name of the 1st column
    field_type  = 'INTEGER'           # column data type
    column_name = 'ingredients'       # name of the column
    field_type_2  = 'TEXT'            # column data type
    create_ingredients_table_query = 'CREATE TABLE {tn} ({nf} {ft}, {cn} {ct})'\
        .format(tn=table_name, nf=id_column, ft=field_type, cn=column_name, ct=field_type_2)
    conn = sqlite3.connect(sqlitefile)
    c = conn.cursor()
    # Create table with ing_id column as primary key
    try:
        c.execute(create_ingredients_table_query)
    except sqlite3.OperationalError as oe:
        print(oe)
    conn.commit()
    conn.close()

def create_scratch_meal_bow_table(sqlitefile):
    table_name  = 'meal_bow'  # name of the table to be created
    id_column   = 'Word' # name of the 1st column
    field_type  = 'TEXT'  # column data type
    column_name = 'Category' # name of the column
    conn = sqlite3.connect(sqlitefile)
    c = conn.cursor()
    try: 
        # Create table with word column as primary key
        c.execute('CREATE TABLE {tn} ({nf} {ft} PRIMARY KEY)'\
                .format(tn=table_name, nf=id_column, ft=field_type))
        # Add a 2nd column named: Keyword
        c.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}"\
                .format(tn=table_name, cn=column_name, ct=field_type))
    except sqlite3.OperationalError as oe:
        print(oe)
    conn.commit()
    conn.close()

def fill_bow(sqlitefile, fraction_list, measure_units, shape_ing, type_ing):
    table_name  = 'meal_bow'  # name of the table to be created
    id_column   = 'Word' # name of the 1st column
    field_type  = 'TEXT'  # column data type
    column_name = 'Category' # name of the column
    conn = sqlite3.connect(sqlitefile)
    c = conn.cursor()
    category    = 'measure_units'
    for record in measure_units:
        c.execute("INSERT OR IGNORE INTO {tn} ({idf}, {cn}) VALUES ('{wv}', '{wk}')".\
            format(tn=table_name, idf=id_column, cn=column_name, wv=record, wk=category))
    category    = 'fraction_list'
    for record in fraction_list:
        c.execute("INSERT OR IGNORE INTO {tn} ({idf}, {cn}) VALUES ('{wv}', '{wk}')".\
            format(tn=table_name, idf=id_column, cn=column_name, wv=record, wk=category))
    category    = 'shape_ing'
    for record in shape_ing:
        c.execute("INSERT OR IGNORE INTO {tn} ({idf}, {cn}) VALUES ('{wv}', '{wk}')".\
            format(tn=table_name, idf=id_column, cn=column_name, wv=record, wk=category))
    category    = 'type_ing'
    for record in type_ing:
        c.execute("INSERT OR IGNORE INTO {tn} ({idf}, {cn}) VALUES ('{wv}', '{wk}')".\
            format(tn=table_name, idf=id_column, cn=column_name, wv=record, wk=category))
    conn.commit()
    conn.close()